<div align="center">
  <img src="https://raw.githubusercontent.com/flyteorg/static-resources/main/flyte/readme/flyte_and_lf.png" alt="Flyte and LF AI & Data Logo" width="250">
</div>

<h1 align="center">
  Flyte School
</h1>

<h3 align="center">
  Module 0: A Practical Introduction to Machine Learning Orchestration
</h3>

This module provides a hands-on introduction to Flyte, an open source machine
learning and data processing orchestration platform. We'll cover the basics of
Flyte and how to use it locally for iterative development.

### Learning Objectives

- 🧱 Understand how tasks, workflows, and launchplans can be used to manage the
  flow of data and models through a sequence of computations.
- 💻 Learn how to spin up a local Flyte cluster on their own workstation and run
  workflows on it for local debugging.
- 🔄 Programmatically run tasks and workflows through the CLI or in a Python
  runtime to quickly iterate on your code.

### Outline

| 🔤 Introduction to Flyte | [45 minutes] |
| --- | --- |
| **Environment Setup** | Setting up your virtual development environment |
| **Flyte Basics** | Tasks, Workflows, and Launch Plans: the building blocks of Flyte |
| **`pyflyte run`** | Run tasks and workflows locally or on a Flyte cluster  |
| **Flyte Console** | A tour of the Flyte console to view workflow progress and status  |
| **`FlyteRemote`** | Programmatically run tasks and workflows  |
| **Scheduling Launch Plans** | Run your workflows on a schedule |
| **Flyte Programming Model** | A mental model of Flyte |

## 🌍 Environment Setup

Follow the instructions in the setup instructions of the [README](./README.md).

We'll be using some environment variables throughout this workshop, so let's
export them right now:

In [26]:
import os
from pathlib import Path

os.environ["FLYTECTL_CONFIG"] = str(Path.home() / ".flyte/config-sandbox.yaml")
os.environ["IMAGE"] = "ghcr.io/unionai-oss/flyte-school:00-intro-latest"

## 🔤 Flyte Basics

Tasks, Workflows, and Launch Plans: the building blocks of Flyte.

### 📦 Tasks as Containerized Functions

A core building block for type-safety, statelessness, and reproducibility.

Let's take a closer look at Flyte tasks.

<image src="static/flyte_tasks.png" width="400px">

We're going to use `flytekit` to write our tasks.

`flytekit` is the Python SDK for Flyte. It's the way data scientists, ML engineers, data engineers, and data analysts write code that will eventually run on a Flyte cluster.

The `@task`-decorated function looks deceptively simple:

In [27]:
from flytekit import task

@task
def hello():
    print("hello world")

hello()

hello world


If we want the task to do anything useful, we want to give it some inputs and
have it produce some outputs.

In [36]:
@task
def square(x: float) -> float:
    return x ** 2

square(x=2.0)

4.0

> ℹ️ In the above code, notice two things:
>
> - **Tasks functions are strongly typed:** this not only provides type safety,
>   it allows Flyte to analyze a sequence of tasks that depend on each other
>   and determine their compatibility.
> - **Tasks arguments must be kwargs:** this is a current constraint of Flyte tasks
>   that may be removed in a future release.

**📝 Exercise**:
1. Try calling `square` with a different data type.
2. Modify the code in the `square` function to output a different data type.

#### 🤔 Why Containerized Functions?

What does type-safety have to do with containers?

Flyte treats tasks as containerized functions. This means that each task runs on
their own isolated container in the Flyte cluster. Coupled with strongly-typed
interfaces, tasks are essentially ✨microservices✨ that can be composed together
to form workflows.

This entails the following benefits:

- **Statelessness:** each task is stateless, which means that it can be run
  multiple times without side effects.
- **Reproducibility:** each task is reproducible, which means that it can be
  run multiple times with the same inputs and produce the same outputs.
- **Portability:** each task is portable, which means that it can be run on
  any Flyte cluster, or any infrastructure that can run containers.
- **Heterogeneity:** each task in a workflow can be written in any language and
  can be run with varying compute requirements.

## 🔀 Workflows and Promises

How Flyte workflows construct an execution graph of tasks.

Now let's take a closer look at Flyte workflows:

<image src="static/flyte_workflows.png" width="500px">

The Flyte `@workflow` is basically a domain-specific language (DSL) that builds an
execution graph that uses tasks as the building blocks for more complex pipelines.

In [71]:
from flytekit import workflow

@task
def error(x: list[float], y: list[float]) -> list[float]:
    return [xi - yi for xi, yi in zip(x, y)]

@task
def squares(x: list[float]) -> list[float]:
    out = [xi ** 2 for xi in x]
    return out

@task
def sum_task(x: list[float]) -> float:
    return sum(x)

@workflow
def sum_of_squares(x: list[float], y: list[float]) -> float:
    errors = error(x=x, y=y)
    squared = squares(x=errors)
    return sum_task(x=squared)


sum_of_squares(x=[1.0, 2.0, 4.0], y=[1.0, 3.0, 6.0])

5.0

**📝 Exercise:**

- In the `sum_of_squares` workflow function, print out the `squared` variable. What
  do you expect to see? What do you actually see?
- Try invoking `sum_of_squares` with different data types.
- Modify one of the types in any of the tasks used in the `sum_of_squares` workflow.

Let's take a look at the [workflows/example_intro.py](./workflows/example_intro.py) script.

In it, you'll see a simple pipeline that uses the penguins dataset to train a
penguin species classifier. This script introduces three core concepts in Flyte:

- `tasks`: the basic unit of compute in Flyte.
- `workflows`: an execution graph of tasks.
- `launchplans`: a mechanism for executing and reusing workflows.

You can run this workflow locally with:

In [43]:
%%sh
python workflows/example_intro.py

DefaultNamedTupleOutput(o0=LogisticRegression(C=0.1, max_iter=5000), o1=0.989010989010989, o2=1.0)


## Iterating on Workflows

### `pyflyte run`

Run tasks and workflows locally or on a Flyte cluster.

Run to run a workflow locally, execute the following command on on your terminal:

In [46]:
%%sh
pyflyte run \
    workflows/example_intro.py training_workflow \
    --hyperparameters '{"C": 0.01}'

DefaultNamedTupleOutput(o0=LogisticRegression(C=0.01, max_iter=2500), o1=0.9743589743589743, o2=1.0)


This is great for the local debugging experience, but what if we want to run this
workflow on an actual Flyte cluster?

`pyflyte run` also supports this use case through the `--remote` flag.

In [48]:
%%sh
pyflyte --config $FLYTECTL_CONFIG \
    run --remote \
    --image $IMAGE \
    workflows/example_intro.py training_workflow \
    --hyperparameters '{"C": 0.01}'

Promise(node:n2.o0)
Go to http://localhost:30080/console/projects/flytesnacks/domains/development/executions/f5f8d11a0f59046d9aa5 to see execution in the console.



Notice how we're providing two extra flags:
- `--config`: this is the path to the Flyte config file, which points `pyflyte run`
  to the Flyte cluster endpoint.
- `--remote`: this flag tells `pyflyte run` that we want to run the workflow on
  a flyte cluster.

Once you execute this command, you should see a message that looks like this:

```
Go to http://localhost:30080/console/projects/flytesnacks/domains/development/executions/<execution_id> to see execution in the console.
```

Where `<execution_id>` is the execution id of the workflow
execution.

**📝 Exercise:**

- Run a task with `pyflyte run` locally.

### 🖥️ Flyte Console

A tour of the Flyte console to view workflow progress and status.

Now let's go to the URL provided by the `pyflyte run` command to see the execution progressing.

The `flyteconsole` is the UI component of the Flyte stack. It provides a way to visualize workflows, launch them from the browser, and obtain useful metadata about Flyte entities and their corresponding executions.

<image src="https://raw.githubusercontent.com/flyteorg/static-resources/main/flytesnacks/getting_started/getting_started_console.gif" width="1000px">

> ℹ️ There are a few things to note about the console:
>
> - It exposes comprehensive metadata about the workflow as it progresses.
> - It visualizes the execution graph of the workflow in the **Graph View**.
> - It profiles the runtime of the workflow in the **Timeline View**.

### 🎮 `FlyteRemote`

Programmatically run tasks and workflows.

You can also run workflows programmatically using the `FlyteRemote` class. This
is useful for:

- 📓 Running Flyte tasks/workflows within a Jupyter notebook
- 🤖 Running Flyte tasks/workflows as a microservice
- 🚢 Integrating Flyte into your CI/CD pipelines, or 

The code below illustrates how we can import the workflow functions into a Python
runtime and execute them on a Flyte cluster.

In [49]:
from workflows import example_intro
from workflows.utils import get_remote

remote = get_remote()
execution = remote.execute_local_workflow(
    example_intro.training_workflow,
    inputs={
        "hyperparameters": example_intro.Hyperparameters(C=0.1, max_iter=5000),
        "test_size": 0.2,
        "random_state": 11,
    }
)
remote.generate_console_url(execution)

'http://localhost:30080/console/projects/flytesnacks/domains/development/executions/f8627c6c5a4f44e40949'

In [50]:
execution = remote.wait(execution)

In [52]:
from sklearn.linear_model import LogisticRegression

clf = execution.outputs.get("o0", LogisticRegression)
clf

LogisticRegression(C=0.1, max_iter=5000)

In [53]:
from palmerpenguins import load_penguins
from workflows import example_intro

data = load_penguins().dropna().sample(5, random_state=123)
features = data[example_intro.FEATURES]
features.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
111,45.6,20.3,191.0,4600.0
158,45.4,14.6,211.0,4800.0
291,50.5,19.6,201.0,4050.0
311,47.5,16.8,199.0,3900.0
186,49.1,14.8,220.0,5150.0


In [54]:
targets = data[example_intro.TARGET]
print("Predictions", clf.predict(features))
print("Ground truth", targets.values)

Predictions ['Adelie' 'Gentoo' 'Chinstrap' 'Chinstrap' 'Gentoo']
Ground truth ['Adelie' 'Gentoo' 'Chinstrap' 'Chinstrap' 'Gentoo']


### ⏱️ Scheduling Launchplans

Run your workflows on a schedule.

A launchplan is a wrapper around a `workflow` function, providing additional
functionality such as scheduling workflows on a particular cadence.

First, let's register our workflow module with `pyflyte register`. This is
a command-line tool for you to package and register python modules that contain
Flyte tasks, workflows, and launchplans.

In [55]:
from datetime import datetime

os.environ["VERSION"] = datetime.now().strftime("%Y%m%d%H%M%S")

In [56]:
%%sh
pyflyte register --image $IMAGE workflows --version $VERSION

Running pyflyte register from /Users/nielsbantilan/git/flyte-school/00-intro with images ImageConfig(default_image=Image(name='default', fqn='ghcr.io/unionai-oss/flyte-school', tag='00-intro-latest'), images=[Image(name='default', fqn='ghcr.io/unionai-oss/flyte-school', tag='00-intro-latest')]) and image destination folder /root on 1 package(s) ('/Users/nielsbantilan/git/flyte-school/00-intro/workflows',)
Registering against localhost:30080
Detected Root /Users/nielsbantilan/git/flyte-school/00-intro, using this to create deployable package...
No output path provided, using a temporary directory at /var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gp/T/tmp9eul9iff instead
Loading packages ['workflows'] under source root /Users/nielsbantilan/git/flyte-school/00-intro
Promise(node:n2.o0)
Successfully serialized 7 flyte objects
[✔] Registration workflows.example_intro.get_data type TASK successful with version 20230720123757
[✔] Registration workflows.example_intro.split_data type TASK successf


Recall in the `example_intro.py` script that we defined a launch plan called
`scheduled_training_workflow`. You can activate scheduled launchplans from the CLI:

First, get the latest launchplan version:

In [57]:
%%sh
flytectl get launchplan \
    --project flytesnacks \
    --domain development \
    scheduled_training_workflow \
    --output yaml --latest  \
    | grep 'version:' -m 1

  version: "20230720123757"


Expected output:
```
version: <version>
```

Using the `flytectl` CLI, activate the launchplan:

In [58]:
%%sh
flytectl update launchplan \
    -p flytesnacks -d development \
    scheduled_training_workflow \
    --version $VERSION --activate

updated launchplan successfully on scheduled_training_workflow

Make sure it's activated:

In [59]:
%%sh
flytectl get launchplan \
    -p flytesnacks -d development \
    scheduled_training_workflow \
    --output yaml --latest \
    | grep ' state:'

  state: ACTIVE


Expected output:
```
state: ACTIVE
``````

You can also use `FlyteRemote` to activate launchplans in Python:

In [60]:
from workflows.utils import get_remote

remote = get_remote()
launch_plan = remote.fetch_launch_plan(name="scheduled_training_workflow")
remote.client.update_launch_plan(launch_plan.id, "ACTIVE")
print("activated scheduled_training_workflow")

activated scheduled_training_workflow


Get the execution for the most recent scheduled run:

In [61]:
recent_executions = [
    ex for ex in remote.recent_executions()
    if ex.spec.launch_plan.name == "scheduled_training_workflow"
]

scheduled_execution = None
if recent_executions:
    scheduled_execution = recent_executions[0]
    scheduled_execution = remote.sync(scheduled_execution)
    
print(scheduled_execution)

<FlyteLiteral id { project: "flytesnacks" domain: "development" name: "f00599e3691638724000" } spec { launch_plan { resource_type: LAUNCH_PLAN project: "flytesnacks" domain: "development" name: "scheduled_training_workflow" version: "20230720123757" } metadata { mode: SCHEDULED scheduled_at { seconds: 1689871200 } system_metadata { } } labels { } annotations { } auth_role { } } closure { phase: RUNNING started_at { seconds: 1689871200 nanos: 84181000 } duration { } created_at { seconds: 1689871200 nanos: 74376000 } updated_at { seconds: 1689871200 nanos: 84181000 } }>


In [62]:
from sklearn.linear_model import LogisticRegression

scheduled_execution = remote.wait(scheduled_execution)
clf = scheduled_execution.outputs.get("o0", LogisticRegression)
clf

LogisticRegression(C=0.1, max_iter=1000)

Deactivate the schedule with the `flytectl` CLI:

In [63]:
%%sh
flytectl update launchplan \
    -p flytesnacks -d development \
    scheduled_training_workflow \
    --version $VERSION --archive

updated launchplan successfully on scheduled_training_workflow

Or with `FlyteRemote`:

In [64]:
remote.client.update_launch_plan(launch_plan.id, "INACTIVE")
print("deactivated scheduled_training_workflow")

deactivated scheduled_training_workflow


## 🛩️ Flyte Programming Model

<image src="static/flyte_programming_model_overview.png" width="800px">

To give you a sense of what's happening when we do `pyflyte run`, let's run just the
`get_data` task from `example_intro.py`:

In [32]:
%%sh
pyflyte run --remote \
    --image $IMAGE \
    workflows/example_intro.py get_data

Go to http://localhost:30080/console/projects/flytesnacks/domains/development/executions/fe1f234aa7f5a49a09f2 to see execution in the console.


If we go to the flyteconsole link provided by the pyflyte run command, we can dig into the guts of a task container execution:

> - In the **Executions** tab, go to the **Logs** link, which will take you to the Kubernetes dashboard logs.
> - If you go to the Pod metadata description, you can see the **Arguments** that are provided as the entrypoint to the task container.

In [33]:
!pyflyte-fast-execute --help

Usage: pyflyte-fast-execute [OPTIONS] [TASK_EXECUTE_CMD]...

  Downloads a compressed code distribution specified by additional-
  distribution and then calls the underlying task execute command for the
  updated code.

Options:
  --additional-distribution TEXT
  --dest-dir TEXT
  --help                          Show this message and exit.


### ⌨️ Type System

The Flyte type system is responsible for a lot of Flyte's production-grade
qualities:

- 👟 Run-time type-checking.
- 📦 Serialization/deserialization of IO between tasks.
- 📝 Type-checking of workflows at compile-time.

This type system is language-agnostic and is implemented in the `flyteidl`
protobuf format. This means that Flyte SDKs can be written in any language.
Currently `flytekit` is the Python implementation, but there are also
Java, Scala, and Javascript SDKs.

<image src="static/flyte_type_system.png" width="400px">

### 🔀 How Data Flows in Flyte

If tasks run in their own containers inside the Flyte cluster, how is data passed between them?

`flytekit` needs to convert all the Python types into something that the `flyteidl`
protobuf type system can understand. This is done by the `flytekit` type engine.
We'll learn more about how this works later, but at a high-level, it looks like
this:

<image src="static/flyte_data_flow.png" width="500px">

To see this diagram in action, let's kick off another workflow execution:

In [11]:
from workflows import example_intro
from workflows.utils import get_remote

remote = get_remote()
execution = remote.execute_local_workflow(
    example_intro.training_workflow,
    inputs={
        "hyperparameters": example_intro.Hyperparameters(C=0.1, max_iter=5000),
        "test_size": 0.2,
        "random_state": 11,
    }
)
remote.generate_console_url(execution)

'http://localhost:30080/console/projects/flytesnacks/domains/development/executions/f29b9a98f19b24330a9a'

Once the execution completes, we can see the inputs and outputs of each task
in the workflow execution.

In [12]:
execution = remote.wait(execution)

Get the node executions:

In [13]:
execution.node_executions.keys()

dict_keys(['end-node', 'n0', 'n1', 'n2', 'n3', 'n4', 'start-node'])

Get the output of the `get_data` node:

In [26]:
get_data_node = execution.node_executions["n0"]
get_data_node.outputs["o0"]._literal_sd.uri

's3://my-s3-bucket/data/tp/f7e7704f07a4a4d84b98-n0-0/8dae9f6bb49d3546d0240fb07baa43e2'

Load the data up using the `remote.remote_context()` context manager.

In [18]:
import pandas as pd

get_data_node = execution.node_executions["n0"]
with remote.remote_context():
    sd = get_data_node.outputs["o0"]
    data = sd.open(pd.DataFrame).all()

data.head(3)

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181.0,3750.0
1,Adelie,39.5,17.4,186.0,3800.0
2,Adelie,40.3,18.0,195.0,3250.0


Get the input of the `split_data` node:

In [19]:
split_data_node = execution.node_executions["n1"]
split_data_node.inputs["data"]._literal_sd.uri

's3://my-s3-bucket/data/w8/f29b9a98f19b24330a9a-n0-0/896e2d26405a8744c6005206da53068b'

Make sure they're the same

In [21]:
assert split_data_node.inputs["data"]._literal_sd.uri == get_data_node.outputs["o0"]._literal_sd.uri
print("Test passed! ✅")

Test passed! ✅


The Flyte sandbox cluster actually runs a Minio object store that stores all of
the inputs and outputs. We can find them on the Minio dashboard http://localhost:30080/minio, or we can get it through the terminal:

In [ ]:
%%sh
export AWS_ACCESS_KEY_ID=minio
export AWS_SECRET_ACCESS_KEY=miniostorage
aws --endpoint-url http://localhost:30002 s3 ls <s3_uri>

## 🌳 Lifecycle of a Workflow

When you run a workflow locally, flytekit just runs the tasks in a Python runtime. However, when you run the workflow on a Flyte cluster, a lot of things are happening under the hood.

`flytepropeller` is the core engine in the Flyte stack that orchestrates:
- ➡️ the execution of tasks in a particular sequence.
- 📊 the management of data dependencies between tasks.
- 💿 the compute infrastructure needed to run a task.
- ... and much more.

This is a very deep topic that we don't have time to cover in this workshop, but
here's a high-level overview of what happens:

<image src="static/flyte_workflow_lifecycle.png" width="500px">

## 🧑‍💻 Development Lifecycle Overview

Typically, working with Flyte as a data scientist looks like this:

- 💻 Create and test tasks/workflows locally with `pyflyte run`
- 📦 Build a container for your tasks/workflows
- 🔁 Iterate on a Flyte cluster with `pyflyte run --remote`
- 🚀 Deploy to production on Flyte cluster using `pyflyte register --non-fast`

We haven't gone through the build step, but it would look something like this:

```bash
make docker-build
```

This make target is in the [`Makefile`](./Makefile), which defines how to build
and push the docker image.

If we go to the [`Dockerfile`](./Dockerfile), we can see that this packages up
all of the Flyte workflow code and the third-party dependencies into the image.

Now let's understand what `pyflyte register` is.

### `pyflyte register`

By default, `pyflyte register` zips up all of the source code of your Flyte
application.

Flyte supports rapid iteration during development via "fast registration" via
`pyflyte register`. This zips up all of the source code of your Flyte 
application and bypasses the need to re-build a docker image with your updated
code in it.

In [65]:
%%sh
pyflyte register \
    --image $IMAGE \
    --project flytesnacks \
    --domain development \
    --version myversion0 \
    workflows

Running pyflyte register from /Users/nielsbantilan/git/flyte-school/00-intro with images ImageConfig(default_image=Image(name='default', fqn='ghcr.io/unionai-oss/flyte-school', tag='00-intro-latest'), images=[Image(name='default', fqn='ghcr.io/unionai-oss/flyte-school', tag='00-intro-latest')]) and image destination folder /root on 1 package(s) ('/Users/nielsbantilan/git/flyte-school/00-intro/workflows',)
Registering against localhost:30080
Detected Root /Users/nielsbantilan/git/flyte-school/00-intro, using this to create deployable package...
No output path provided, using a temporary directory at /var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gp/T/tmpjkdpsihu instead
Computed version is acp5-WGxARLHZgGZA4FtSg==
Loading packages ['workflows'] under source root /Users/nielsbantilan/git/flyte-school/00-intro
Promise(node:n2.o0)
Successfully serialized 7 flyte objects
[✔] Registration workflows.example_intro.get_data type TASK successful with version acp5-WGxARLHZgGZA4FtSg==
[✔] Registratio


> ℹ️ **Note**: You can provide an explicit `--version` flag, but by default `flytekit` will create a version string for you.

Now let's register a new version of the `example_intro.py` workflow with `pyflyte run`:

In [96]:
%%sh
pyflyte run --remote \
    --image $IMAGE \
    workflows/example_intro.py training_workflow \
    --hyperparameters '{"C": 0.01}'

Go to http://localhost:30080/console/projects/flytesnacks/domains/development/executions/f3659537700944e469b8 to see execution in the console.



If we go to the Flyte console link provided the `pyflyte run`, we can understand
what happened during the execution by going to the **Task** tag and looking at
the `--additional-distribution` flag in the container arguments. The `tar.gz` file
is the packaged-up Flyte code in our repository, which `flytekit` understands
is the actual task code that we want to run.

### `pyflyte register --non-fast`

Unlike `pyflyte register`, `pyflyte register --non-fast` *will not* package up
the user code and simply use the source code that is present in the image when
it was originally built.

This is the **recommended** way to deploy to production because it ensures that
the source code that is running in production is the same as the source code in
the image, since the `tar.gz` *could in theory* be swapped out by a malicious
actor.

Since we're deploying to production, let's specify the `--domain production` flag.

In [69]:
%%sh
pyflyte register \
    --image $IMAGE \
    --project flytesnacks \
    --domain production \
    --non-fast \
    --version prod-v0 \
    workflows

Running pyflyte register from /Users/nielsbantilan/git/flyte-school/00-intro with images ImageConfig(default_image=Image(name='default', fqn='ghcr.io/unionai-oss/flyte-school', tag='00-intro-latest'), images=[Image(name='default', fqn='ghcr.io/unionai-oss/flyte-school', tag='00-intro-latest')]) and image destination folder /root on 1 package(s) ('/Users/nielsbantilan/git/flyte-school/00-intro/workflows',)
Registering against localhost:30080
Detected Root /Users/nielsbantilan/git/flyte-school/00-intro, using this to create deployable package...
Loading packages ['workflows'] under source root /Users/nielsbantilan/git/flyte-school/00-intro
Successfully serialized 7 flyte objects
[✔] Registration workflows.example_intro.get_data type TASK successful with version prod-v0
[✔] Registration workflows.example_intro.split_data type TASK successful with version prod-v0
[✔] Registration workflows.example_intro.train_model type TASK successful with version prod-v0
[✔] Registration workflows.exampl

## Conclusion

🎉 Congrats! 🎉

You've made it through to the end of this module! To recap, we've:

- 🧱 Learned the basics constructs of Flyte: tasks, workflows, and launchplans.
- 🔀 Understood how Flyte orchestrates execution graphs, data, and compute infrastructure.
- ⌨️ Understood how the Flyte type system provides type-safety as well as IO abstraction.
- 👟 Learned how to run tasks/workflows programmatically through the `pyflyte run` CLI and Python `FlyteRemote` client.

### Resources

<a href="https://docs.flyte.org/en/latest/">
    <img src="https://img.shields.io/badge/Flyte-Docs-purple?style=for-the-badge" alt="Flyte Docs" />
</a>
<a href="https://slack.flyte.org">
    <img src="https://img.shields.io/badge/Slack-Chat-pink?style=for-the-badge&logo=slack" alt="Flyte Slack" />
</a>
<a href="https://github.com/flyteorg/flyte">
    <img src="https://img.shields.io/badge/Flyte-Github%20Repo-black?style=for-the-badge&logo=github" alt="Flyte Repo" />
</a>

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vRpMyFw3lfocwbDXcT3XhLEROjZ5mYVCrmQ5WHxikX4m6tlf00q8FgDgwFZSUwheM0luy-KR6Djw8yz/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>